 # BRAIN WALL
 

In [ ]:
"""
Hole in the Wall - Juego de Poses con Visión por Computador
Requisitos: pip install opencv-python mediapipe pygame numpy
"""

import cv2
import mediapipe as mp
import pygame
import numpy as np
import random
import time

# Configuración
WINDOW_WIDTH = 1280
WINDOW_HEIGHT = 720
FPS = 30

# Colores
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (100, 150, 255)
GREEN = (100, 255, 100)
RED = (255, 100, 100)
YELLOW = (255, 255, 100)
PURPLE = (180, 100, 255)

# Poses predefinidas (puntos clave de MediaPipe normalizados)
# Cada pose es un conjunto de conexiones que forman la silueta
POSES = [
    {
        "name": "T-Pose",
        "description": "Brazos extendidos horizontalmente",
        "keypoints": {
            11: (0.2, 0.4),   # hombro izq
            12: (0.8, 0.4),   # hombro der
            13: (0.05, 0.4),  # codo izq
            14: (0.95, 0.4),  # codo der
            15: (0.0, 0.4),   # muñeca izq
            16: (1.0, 0.4),   # muñeca der
            23: (0.35, 0.7),  # cadera izq
            24: (0.65, 0.7),  # cadera der
            25: (0.35, 0.85), # rodilla izq
            26: (0.65, 0.85), # rodilla der
        }
    },
    {
        "name": "Estrella",
        "description": "Brazos y piernas abiertas",
        "keypoints": {
            11: (0.25, 0.35),
            12: (0.75, 0.35),
            13: (0.1, 0.25),
            14: (0.9, 0.25),
            15: (0.0, 0.15),
            16: (1.0, 0.15),
            23: (0.35, 0.6),
            24: (0.65, 0.6),
            25: (0.2, 0.8),
            26: (0.8, 0.8),
            27: (0.1, 0.95),
            28: (0.9, 0.95),
        }
    },
    {
        "name": "Brazos Arriba",
        "description": "Levanta ambos brazos",
        "keypoints": {
            11: (0.35, 0.4),
            12: (0.65, 0.4),
            13: (0.3, 0.25),
            14: (0.7, 0.25),
            15: (0.25, 0.1),
            16: (0.75, 0.1),
            23: (0.4, 0.7),
            24: (0.6, 0.7),
            25: (0.4, 0.85),
            26: (0.6, 0.85),
        }
    },
    {
        "name": "Pierna Izquierda",
        "description": "Levanta la pierna izquierda",
        "keypoints": {
            11: (0.35, 0.4),
            12: (0.65, 0.4),
            13: (0.25, 0.5),
            14: (0.75, 0.5),
            15: (0.2, 0.6),
            16: (0.8, 0.6),
            23: (0.4, 0.65),
            24: (0.6, 0.65),
            25: (0.25, 0.55),
            26: (0.6, 0.85),
            27: (0.15, 0.65),
            28: (0.6, 0.98),
        }
    },
    {
        "name": "Agachado",
        "description": "Agáchate con brazos al frente",
        "keypoints": {
            11: (0.35, 0.5),
            12: (0.65, 0.5),
            13: (0.2, 0.45),
            14: (0.8, 0.45),
            15: (0.1, 0.4),
            16: (0.9, 0.4),
            23: (0.35, 0.7),
            24: (0.65, 0.7),
            25: (0.3, 0.85),
            26: (0.7, 0.85),
        }
    },
    {
        "name": "Diagonal",
        "description": "Brazo izquierdo arriba, derecho abajo",
        "keypoints": {
            11: (0.35, 0.4),
            12: (0.65, 0.4),
            13: (0.2, 0.25),
            14: (0.8, 0.55),
            15: (0.1, 0.1),
            16: (0.9, 0.7),
            23: (0.4, 0.7),
            24: (0.6, 0.7),
            25: (0.4, 0.85),
            26: (0.6, 0.85),
        }
    },
]

# Conexiones del cuerpo para dibujar
BODY_CONNECTIONS = [
    (11, 12),  # hombros
    (11, 13), (13, 15),  # brazo izq
    (12, 14), (14, 16),  # brazo der
    (11, 23), (12, 24),  # torso
    (23, 24),  # caderas
    (23, 25), (25, 27),  # pierna izq
    (24, 26), (26, 28),  # pierna der
]


class HoleInTheWallGame:
    def __init__(self):
        pygame.init()
        pygame.display.set_caption("Hole in the Wall - ¡Haz la Pose!")
        self.screen = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
        self.clock = pygame.time.Clock()
        
        # MediaPipe
        self.mp_pose = mp.solutions.pose
        self.pose = self.mp_pose.Pose(
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )
        
        # Cámara
        self.cap = cv2.VideoCapture(0)
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        
        # Estado del juego
        self.score = 0
        self.lives = 3
        self.level = 1
        self.current_pose = None
        self.wall_position = WINDOW_WIDTH
        self.wall_speed = 3
        self.time_per_wall = 5.0
        self.wall_start_time = time.time()
        self.game_state = "menu"  # menu, playing, gameover
        self.last_match_score = 0
        self.show_result = False
        self.result_time = 0
        
        # Landmarks detectados
        self.detected_landmarks = {}
        
        # Fuentes
        self.font_large = pygame.font.Font(None, 72)
        self.font_medium = pygame.font.Font(None, 48)
        self.font_small = pygame.font.Font(None, 36)
        
        self.select_new_pose()
    
    def select_new_pose(self):
        self.current_pose = random.choice(POSES)
        self.wall_position = WINDOW_WIDTH
        self.wall_start_time = time.time()
        self.show_result = False
    
    def process_camera(self):
        ret, frame = self.cap.read()
        if not ret:
            return None
        
        frame = cv2.flip(frame, 1)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.pose.process(rgb_frame)
        
        self.detected_landmarks = {}
        if results.pose_landmarks:
            for idx, landmark in enumerate(results.pose_landmarks.landmark):
                if landmark.visibility > 0.5:
                    self.detected_landmarks[idx] = (landmark.x, landmark.y)
        
        # Convertir para Pygame
        frame_surface = pygame.surfarray.make_surface(
            np.rot90(rgb_frame)
        )
        frame_surface = pygame.transform.scale(frame_surface, (480, 360))
        return frame_surface
    
    def calculate_match_score(self):
        if not self.current_pose or not self.detected_landmarks:
            return 0
        
        target_keypoints = self.current_pose["keypoints"]
        total_distance = 0
        matched_points = 0
        
        for idx, target_pos in target_keypoints.items():
            if idx in self.detected_landmarks:
                detected_pos = self.detected_landmarks[idx]
                distance = np.sqrt(
                    (detected_pos[0] - target_pos[0])**2 + 
                    (detected_pos[1] - target_pos[1])**2
                )
                total_distance += distance
                matched_points += 1
        
        if matched_points == 0:
            return 0
        
        avg_distance = total_distance / matched_points
        score = max(0, 100 - (avg_distance * 200))
        return int(score)
    
    def draw_wall_silhouette(self):
        if not self.current_pose:
            return
        
        # Calcular posición y tamaño de la pared
        wall_width = 350
        wall_height = 500
        wall_x = self.wall_position
        wall_y = (WINDOW_HEIGHT - wall_height) // 2
        
        # Dibujar fondo de la pared
        wall_color = BLUE if self.wall_position > 200 else YELLOW
        pygame.draw.rect(self.screen, wall_color, 
                        (wall_x, wall_y, wall_width, wall_height))
        pygame.draw.rect(self.screen, WHITE, 
                        (wall_x, wall_y, wall_width, wall_height), 3)
        
        # Dibujar silueta (hueco)
        keypoints = self.current_pose["keypoints"]
        scaled_points = {}
        
        for idx, (x, y) in keypoints.items():
            px = wall_x + int(x * wall_width)
            py = wall_y + int(y * wall_height)
            scaled_points[idx] = (px, py)
        
        # Dibujar conexiones de la silueta
        for conn in BODY_CONNECTIONS:
            if conn[0] in scaled_points and conn[1] in scaled_points:
                pygame.draw.line(self.screen, BLACK,
                               scaled_points[conn[0]], 
                               scaled_points[conn[1]], 20)
        
        # Dibujar puntos
        for idx, pos in scaled_points.items():
            pygame.draw.circle(self.screen, BLACK, pos, 15)
    
    def draw_player_skeleton(self):
        if not self.detected_landmarks:
            return
        
        # Área del jugador (izquierda de la pantalla)
        player_x = 50
        player_y = 110
        player_w = 480
        player_h = 360
        
        # Dibujar conexiones
        for conn in BODY_CONNECTIONS:
            if conn[0] in self.detected_landmarks and conn[1] in self.detected_landmarks:
                p1 = self.detected_landmarks[conn[0]]
                p2 = self.detected_landmarks[conn[1]]
                
                x1 = player_x + int(p1[0] * player_w)
                y1 = player_y + int(p1[1] * player_h)
                x2 = player_x + int(p2[0] * player_w)
                y2 = player_y + int(p2[1] * player_h)
                
                pygame.draw.line(self.screen, GREEN, (x1, y1), (x2, y2), 5)
        
        # Dibujar puntos
        for idx, (x, y) in self.detected_landmarks.items():
            px = player_x + int(x * player_w)
            py = player_y + int(y * player_h)
            pygame.draw.circle(self.screen, YELLOW, (px, py), 8)
    
    def draw_ui(self):
        # Puntuación
        score_text = self.font_medium.render(f"Puntos: {self.score}", True, WHITE)
        self.screen.blit(score_text, (20, 20))
        
        # Vidas
        lives_text = self.font_medium.render(f"Vidas: {'❤️ ' * self.lives}", True, RED)
        self.screen.blit(lives_text, (20, 70))
        
        # Nivel
        level_text = self.font_small.render(f"Nivel: {self.level}", True, WHITE)
        self.screen.blit(level_text, (WINDOW_WIDTH - 150, 20))
        
        # Nombre de la pose
        if self.current_pose:
            pose_text = self.font_medium.render(
                f"Pose: {self.current_pose['name']}", True, YELLOW)
            self.screen.blit(pose_text, (WINDOW_WIDTH//2 - 100, 20))
            
            desc_text = self.font_small.render(
                self.current_pose['description'], True, WHITE)
            self.screen.blit(desc_text, (WINDOW_WIDTH//2 - 120, 65))
        
        # Match score en tiempo real
        match = self.calculate_match_score()
        color = GREEN if match > 70 else YELLOW if match > 40 else RED
        match_text = self.font_medium.render(f"Match: {match}%", True, color)
        self.screen.blit(match_text, (WINDOW_WIDTH - 200, 60))
        
        # Barra de tiempo
        elapsed = time.time() - self.wall_start_time
        remaining = max(0, self.time_per_wall - elapsed)
        bar_width = int((remaining / self.time_per_wall) * 300)
        pygame.draw.rect(self.screen, (50, 50, 50), (WINDOW_WIDTH//2 - 150, WINDOW_HEIGHT - 50, 300, 25))
        pygame.draw.rect(self.screen, GREEN if remaining > 2 else RED, 
                        (WINDOW_WIDTH//2 - 150, WINDOW_HEIGHT - 50, bar_width, 25))
        
        # Resultado temporal
        if self.show_result and time.time() - self.result_time < 1.5:
            if self.last_match_score >= 60:
                result_text = self.font_large.render(f"¡BIEN! +{self.last_match_score}", True, GREEN)
            else:
                result_text = self.font_large.render("¡FALLASTE!", True, RED)
            self.screen.blit(result_text, (WINDOW_WIDTH//2 - 150, WINDOW_HEIGHT//2))
    
    def draw_menu(self):
        self.screen.fill((30, 30, 50))
        
        title = self.font_large.render("HOLE IN THE WALL", True, YELLOW)
        self.screen.blit(title, (WINDOW_WIDTH//2 - 220, 150))
        
        subtitle = self.font_medium.render("¡Haz las poses para pasar!", True, WHITE)
        self.screen.blit(subtitle, (WINDOW_WIDTH//2 - 180, 230))
        
        instructions = [
            "📷 Asegúrate de que la cámara te vea completo",
            "🏃 Imita la silueta antes de que llegue la pared",
            "⭐ Consigue más del 60% de match para pasar",
            "",
            "Presiona ESPACIO para comenzar",
            "Presiona ESC para salir"
        ]
        
        for i, line in enumerate(instructions):
            text = self.font_small.render(line, True, WHITE)
            self.screen.blit(text, (WINDOW_WIDTH//2 - 220, 320 + i * 45))
    
    def draw_gameover(self):
        self.screen.fill((50, 30, 30))
        
        title = self.font_large.render("GAME OVER", True, RED)
        self.screen.blit(title, (WINDOW_WIDTH//2 - 150, 200))
        
        score_text = self.font_medium.render(f"Puntuación Final: {self.score}", True, WHITE)
        self.screen.blit(score_text, (WINDOW_WIDTH//2 - 150, 300))
        
        level_text = self.font_medium.render(f"Nivel Alcanzado: {self.level}", True, WHITE)
        self.screen.blit(level_text, (WINDOW_WIDTH//2 - 140, 360))
        
        restart_text = self.font_small.render("Presiona ESPACIO para reiniciar", True, YELLOW)
        self.screen.blit(restart_text, (WINDOW_WIDTH//2 - 180, 450))
    
    def update(self):
        if self.game_state != "playing":
            return
        
        # Mover pared
        self.wall_position -= self.wall_speed
        
        # Verificar si la pared llegó
        if self.wall_position <= 100:
            match_score = self.calculate_match_score()
            self.last_match_score = match_score
            self.show_result = True
            self.result_time = time.time()
            
            if match_score >= 60:
                self.score += match_score
                if self.score > self.level * 300:
                    self.level += 1
                    self.wall_speed = min(8, 3 + self.level * 0.5)
                    self.time_per_wall = max(3.0, 5.0 - self.level * 0.3)
            else:
                self.lives -= 1
                if self.lives <= 0:
                    self.game_state = "gameover"
                    return
            
            self.select_new_pose()
    
    def reset_game(self):
        self.score = 0
        self.lives = 3
        self.level = 1
        self.wall_speed = 3
        self.time_per_wall = 5.0
        self.select_new_pose()
        self.game_state = "playing"
    
    def run(self):
        running = True
        
        while running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                elif event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_ESCAPE:
                        if self.game_state == "playing":
                            self.game_state = "menu"
                        else:
                            running = False
                    elif event.key == pygame.K_SPACE:
                        if self.game_state == "menu":
                            self.reset_game()
                        elif self.game_state == "gameover":
                            self.reset_game()
            
            # Procesar cámara
            camera_surface = self.process_camera()
            
            # Dibujar según estado
            if self.game_state == "menu":
                self.draw_menu()
                if camera_surface:
                    self.screen.blit(camera_surface, (WINDOW_WIDTH - 500, WINDOW_HEIGHT - 380))
            
            elif self.game_state == "playing":
                self.screen.fill((20, 20, 40))
                
                # Dibujar cámara de fondo
                if camera_surface:
                    self.screen.blit(camera_surface, (50, 110))
                    pygame.draw.rect(self.screen, WHITE, (50, 110, 480, 360), 2)
                
                self.draw_player_skeleton()
                self.draw_wall_silhouette()
                self.draw_ui()
                self.update()
            
            elif self.game_state == "gameover":
                self.draw_gameover()
            
            pygame.display.flip()
            self.clock.tick(FPS)
        
        self.cap.release()
        self.pose.close()
        pygame.quit()


if __name__ == "__main__":
    game = HoleInTheWallGame()
    game.run()

ModuleNotFoundError: No module named 'cv2'